In [1]:
"""Model Comparison for Handball Match Prediction
Comparison of various Bayesian and frequentist models for sports outcome prediction
"""

import numpy as np
import pandas as pd

from ssat.bayesian import (
    NegBinom,
    Poisson,
    PoissonDecay,
    Skellam,
    SkellamDecay,
    SkellamZero,
)
from ssat.data import get_sample_handball_match_data
from ssat.frequentist import GSSD, PRP, TOOR, ZSD, BradleyTerry
from ssat.metrics import (
    average_rps,
    balanced_accuracy,
    calibration_error,
    ignorance_score,
    multiclass_brier_score,
    multiclass_log_loss,
)
from ssat.utils import dixon_coles_weights


In [2]:
# Configuration
np.random.seed(42)
LEAGUE = "Starligue"
SEASONS = [2024, 2025]
TRAIN_SPLIT = 0.8


In [3]:
# Load and filter data
df = get_sample_handball_match_data()
print(f"Available leagues: {list(df.league.unique())}")

match_df = df.loc[(df["league"] == LEAGUE) & (df["season"].isin(SEASONS))]
print(f"Dataset size: {len(match_df)} matches")


Available leagues: ['European Championship', 'Kvindeligaen Women', 'Handbollsligan Women', 'Herre Handbold Ligaen', 'Liga ASOBAL', 'Starligue', 'EHF Euro Cup']
Dataset size: 258 matches


In [4]:
# Data preparation
goal_diff = match_df["home_goals"] - match_df["away_goals"]
outcomes = np.sign(goal_diff).replace(
    {-1: 2, 0: 1, 1: 0}
)  # 0=Home win, 1=Draw, 2=Away win

X = match_df[["home_team", "away_team"]]
Z = match_df[["home_goals", "away_goals"]]
y = goal_diff
dt = match_df["datetime"]

# Train-test split
train_size = int(len(match_df) * TRAIN_SPLIT)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
Z_train, Z_test = Z[:train_size], Z[train_size:]
dt_train, dt_test = dt[:train_size], dt[train_size:]
outcomes_test = outcomes[train_size:]

weights_train = dixon_coles_weights(dt_train)

print(f"Training set: {len(X_train)} matches")
print(f"Test set: {len(X_test)} matches")


Training set: 206 matches
Test set: 52 matches


In [5]:
# Initialize models
models = [
    ("Bradley-Terry", BradleyTerry()),
    ("PRP", PRP()),
    ("GSSD", GSSD()),
    ("TOOR", TOOR()),
    ("ZSD", ZSD()),
    ("Poisson", Poisson()),
    ("Negative Binomial", NegBinom()),
    ("Skellam", Skellam()),
    ("Skellam Zero", SkellamZero()),
    ("Skellam Decay", SkellamDecay()),
    ("Poisson Decay", PoissonDecay()),
]


In [6]:
# Model evaluation
results = []

for name, model in models:
    print(f"Training {name}...")

    try:
        model.fit(X=X_train, y=y_train, Z=Z_train, weights=weights_train)
    except (
        Exception
    ):  # Poisson and Negative Binomial fits on home and away goals separately
        model.fit(X=X_train, y=Z_train, Z=Z_train, weights=weights_train)

    preds_proba = model.predict_proba(X_test)

    # Calculate metrics
    metrics = {
        "Model": name,
        "Brier Score": multiclass_brier_score(outcomes_test, preds_proba),
        "Log Loss": multiclass_log_loss(outcomes_test, preds_proba),
        "RPS": average_rps(outcomes_test, preds_proba),
        "Calibration Error": calibration_error(outcomes_test, preds_proba),
        "Ignorance Score": ignorance_score(outcomes_test, preds_proba),
        "Balanced Accuracy": balanced_accuracy(outcomes_test, preds_proba),
    }
    results.append(metrics)


Training Bradley-Terry...
Training PRP...
Training GSSD...
Training TOOR...
Training ZSD...
Training Poisson...


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

                                                                                                                                                                
Training Negative Binomial...


chain 1 |          | 00:00 Status

...

chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

                                                                                                                                                                
Training Poisson Decay...


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

In [7]:
# Display results
results_df = pd.DataFrame(results).set_index("Model")
print("\nModel Performance Comparison:")
print("=" * 50)
results_df.round(4)



Model Performance Comparison:


,Brier Score,Log Loss,RPS,Calibration Error,Ignorance Score,Balanced Accuracy
Model,,,,,,
Bradley-Terry,0.1550,0.7900,0.1885,0.1503,1.1397,0.4882
PRP,0.1542,0.7802,0.1872,0.0821,1.1256,0.4490
GSSD,0.1650,0.8257,0.2027,0.1141,1.1913,0.4268
TOOR,0.1523,0.7733,0.1845,0.1146,1.1156,0.4993
ZSD,0.1541,0.7801,0.1869,0.0947,1.1254,0.4379
Poisson,0.1593,0.8379,0.1944,0.1099,1.2089,0.4601
Negative Binomial,0.1663,0.8830,0.2042,0.0854,1.2739,0.4797
Skellam,0.1542,0.7751,0.1860,0.1819,1.1183,0.4797
Skellam Zero,0.1547,0.7776,0.1867,0.2011,1.1219,0.4797


In [8]:
# Performance ranking (lower is better for most metrics, higher for accuracy)
ranking_metrics = [
    "Brier Score",
    "Log Loss",
    "RPS",
    "Calibration Error",
    "Ignorance Score",
]
accuracy_metrics = ["Balanced Accuracy"]

print("\nTop 3 Models by Metric:")
print("=" * 30)

for metric in ranking_metrics:
    top_3 = results_df[metric].nsmallest(3)
    print(f"\n{metric}:")
    for i, (model, score) in enumerate(top_3.items(), 1):
        print(f"  {i}. {model}: {score:.4f}")

for metric in accuracy_metrics:
    top_3 = results_df[metric].nlargest(3)
    print(f"\n{metric}:")
    for i, (model, score) in enumerate(top_3.items(), 1):
        print(f"  {i}. {model}: {score:.4f}")



Top 3 Models by Metric:

Brier Score:
  1. TOOR: 0.1523
  2. ZSD: 0.1541
  3. Skellam: 0.1542

Log Loss:
  1. TOOR: 0.7733
  2. Skellam: 0.7751
  3. Skellam Zero: 0.7776

RPS:
  1. TOOR: 0.1845
  2. Skellam: 0.1860
  3. Skellam Zero: 0.1867

Calibration Error:
  1. PRP: 0.0821
  2. Negative Binomial: 0.0854
  3. ZSD: 0.0947

Ignorance Score:
  1. TOOR: 1.1156
  2. Skellam: 1.1183
  3. Skellam Zero: 1.1219

Balanced Accuracy:
  1. TOOR: 0.4993
  2. Bradley-Terry: 0.4882
  3. Negative Binomial: 0.4797
